# Домашнее задание 4

## Реализация градиентного бустинга

В рамках этой задачи нужно написать градиентный бустинг над решающими деревьями в задаче классификации. В качестве функции потерь предлагается взять **log loss**. Про рего можно прочитать подробнее здесь: https://scikit-learn.org/stable/modules/model_evaluation.html#log-loss


$y_i$ это правильный ответ (0 или 1), $\hat{y}_i$ это ваше предскзаание

Может показаться, что надо максимизировать функцию $L(\hat{y}, y) = \sum_{i=1}^n y_i \log(\hat{y}_i) + (1 - y_i) \log(1 - \hat{y}_i)$, где $y_i$

Да, но нет. Лучше максимизировать функцию $L(\hat{y}, y) = \sum_{i=1}^n y_i \log(f(\hat{y}_i)) + (1 - y_i) \log(1 - f(\hat{y}_i))$, где $f(x) = \frac{1}{1 + e^{-x}}$. Благодаря этому у вас не будет ограничений на принимаеммые значения для $\hat{y}_i$

### Задание 1

Напишите вычисление производной f(x), обычно её называют **сигмоида**.

In [ ]:
import numpy as np

In [ ]:
def sigmoid(x):
    return 1. / (1 + np.exp(-x))


def der_sigmoid(x):
    return None # TODO

In [ ]:
der_sigmoid(0) == 0.25

In [ ]:
der_sigmoid(np.array([0, 0])) == np.array([0.25, 0.25])

In [ ]:
der_sigmoid(np.log(3)) == 0.1875

**Значение для формы:**

In [ ]:
print(round(der_sigmoid(np.array([-10, 4.1, -1, 2])).sum() + sigmoid(0.42), 4))

Хорошо, теперь мы умеем считать производную функции f, но надо найти производную log loss-а по $\hat{y}$ в первом варианте 

Напоминание, первый вариант это  $y_i \log(\hat{y}_i) + (1 - y_i) \log(1 - \hat{y}_i)$

### Задание 2

Напишите вычисление производной log loss-a

In [ ]:
def der_log_loss(y_hat, y_true):
    """
    0 < y_hat < 1
    """
    return None # TODO

In [ ]:
der_log_loss(0.5, 0) == -2

In [ ]:
der_log_loss(0.5, 1) == 2

In [ ]:
der_log_loss(np.array([0.8, 0.8]), np.array([1, 1])) == np.array([1.25, 1.25])

**Значение для формы**

In [ ]:
print(round(-sum(der_log_loss((x + 1) / 100., x % 2) for x in range(99)), 2))

Отлично теперь мы можем воспользоваться производной сложной функции и получить вычисление градиента формулы по второму варианту:

In [ ]:
def calc_gradient(y_hat, y_true):
    return der_log_loss(sigmoid(y_hat), y_true) * der_sigmoid(y_hat)

Теперь мы можем написать код градиентного бустинга для классификации

### Задание 3

Допишите класс

In [ ]:
from sklearn.base import BaseEstimator # чтобы поддержать интерфейс sklearn
from sklearn.tree import DecisionTreeRegressor # для обучения на каждой итерации

In [ ]:
class SimpleGB(BaseEstimator):
    def __init__(self, tree_params_dict, iters=100, tau=1e-1):
        """
        tree_params_dict - словарь параметров, которые надо использовать при обучении дерева на итерации
        iters - количество итераций
        tau - коэффициент перед предсказаниями деревьев на каждой итерации
        """
        self.tree_params_dict = tree_params_dict
        self.iters = iters
        self.tau = tau
        
    def fit(self, X_data, y_data):
        self.estimators = []
        curr_pred = 0
        for iter_num in range(self.iters):
            # Нужно найти градиент функции потерь по предсказниям в точке curr_pred
            grad = None # TODO
            # Мы максимизируем, поэтому надо обучить DecisionTreeRegressor с параметрами 
            # tree_params_dict по X_data предсказывать grad
            algo = None # TODO
            self.estimators.append(algo)
            # все предсказания домножаются на tau и обновляется переменная curr_pred
            curr_pred += self.tau * algo.predict(X_data)
    
    def predict(self, X_data):
        # изначально все предскзания нули
        res = np.zeros(X_data.shape[0])
        for estimator in self.estimators:
            # нужно сложить все предсказания деревьев с весом self.tau
            pass # TODO
            
        return (res > 0).astype(int)

## Проверка качества полученного класса (в самом низу код для формы)

Можете поиграться с параметрами, посмотрим, у кого самое лучшее качество получится

In [ ]:
# для оценки качества
from sklearn.model_selection import cross_val_score

# для генерации датасетов
from sklearn.datasets import make_classification

# для сравнения
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier

import warnings
warnings.filterwarnings('ignore')

In [ ]:
X_data, y_data = make_classification(n_samples=1000, n_features=10, random_state=42)

In [ ]:
algo = SimpleGB(
    tree_params_dict={
        'max_depth':4
    },
    iters=100,
    tau = 0.1
)

In [ ]:
np.mean(cross_val_score(algo, X_data, y_data, cv=5, scoring='accuracy'))

In [ ]:
np.mean(cross_val_score(DecisionTreeClassifier(), X_data, y_data, cv=5, scoring='accuracy'))

In [ ]:
np.mean(cross_val_score(XGBClassifier(), X_data, y_data, cv=5, scoring='accuracy'))

In [ ]:
np.mean(cross_val_score(LogisticRegression(), X_data, y_data, cv=5, scoring='accuracy'))

**Значение для формы**

In [ ]:
from sklearn.model_selection import StratifiedKFold

print(round(np.mean(cross_val_score(SimpleGB(
    tree_params_dict={
        'max_depth': 4
    },
    iters=1000,
    tau = 0.01
), X_data, y_data, cv=StratifiedKFold(4, random_state=42), scoring='accuracy')), 3))